In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd

from pathlib import Path
URL = "https://sigel.aneel.gov.br/arcgis/rest/services/PORTAL/WFS/MapServer/0/query"

In [3]:
def get_data(save_path="outputs/response.geojson"):
    """Extrai dados de aerogeradores do endpoint da ANEEL e salva em GeoJSON."""
    params = {
        "where": "1=1",       # condição obrigatória
        "outFields": "*",     # todas as colunas
        "f": "geojson"        # saída em GeoJSON
    }

    # Faz a requisição
    resp = requests.get(URL, params=params)
    resp.raise_for_status()  # erro se não retornar 200
    
    # Salva em arquivo
    Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(resp.json(), f, ensure_ascii=False, indent=2)

    print(f"Dados salvos em {save_path}")
    return save_path

In [5]:
def process_geodata(input_path="outputs/response.geojson", output_path="outputs/aerogeradores.csv"):
    """Carrega GeoJSON, adiciona lat/lon e exporta para CSV."""

    # Carrega o GeoJSON em um GeoDataFrame
    gdf = gpd.read_file(input_path)

    # Adicionar lat/lon
    gdf["lat"] = gdf.geometry.y
    gdf["lon"] = gdf.geometry.x

    # Normalizar colunas para snake_case
    gdf.columns = [c.strip().lower().replace(" ", "_") for c in gdf.columns]

    # Remover duplicatas
    gdf = gdf.drop_duplicates()

    # Exportar CSV
    gdf.to_csv(output_path, index=False)
    print(f"CSV salvo em {output_path}")

    return gdf

In [7]:
# Etapa 1 - Baixar dados
geojson_path = get_data()

# Etapa 2 - Processar dados
gdf = process_geodata(geojson_path)
gdf.head()

Dados salvos em outputs/response.geojson
CSV salvo em outputs/aerogeradores.csv


,pot_mw,alt_total,alt_torre,diam_rotor,data_atualizacao,eol_versao_id,nome_eol,den_aeg,x,y,...,operacao,fuso_ag,proprietario,origem,objectid,uf,ceg,geometry,lat,lon
0,1.85,121.25,80.0,82.5,1666625847000,30739,Caititu,CAI-083,754125.00,8398879.00,...,Sim,UTM 23 S (MER -45),Central Eólica Caititu S.A.,None,1,BA,EOLCVBA030739-4-01,POINT (-42.64249 -14.47097),-14.470968,-42.642495
1,2.00,145.00,95.0,100.0,1666625847000,30512,Dreen Boa Vista,BV07,175728.00,9433802.00,...,Sim,UTM 25 S (MER -33),GE Boa Vista S.A.,None,2,RN,EOLCVRN030512-0-01,POINT (-35.92447 -5.11575),-5.115754,-35.924469
2,2.00,145.00,95.0,100.0,1666625847000,30497,Renascença V,15,184692.00,9417056.00,...,Sim,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,None,3,RN,EOLCVRN030497-2-01,POINT (-35.84437 -5.26742),-5.267424,-35.844373
3,2.70,150.00,89.0,122.0,1666625847000,30864,SM,1,176028.00,9397684.00,...,Sim,UTM 25 S (MER -33),Santa Maria Energias Renováveis S.A.,None,4,RN,EOLCVRN030864-1-01,POINT (-35.92329 -5.44208),-5.442082,-35.923294
4,2.10,143.70,96.2,95.0,1666625847000,30295,Colônia,AEG-08,513347.87,9608681.26,...,Sim,UTM 24 S (MER -39),Central Geradora Eólica Colônia S.A.,None,5,CE,EOLCVCE030295-3-01,POINT (-38.87983 -3.54033),-3.540330,-38.879826
